In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.utils import to_categorical
import cv2
import os
import json
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from sklearn.model_selection import train_test_split

In [22]:
coco = COCO('/Users/wiesruyters/Documents/WhD/Repositories/NOS_scrape/Datasets/Annotated_img_politicians/result.json')
categories = coco.loadCats(coco.getCatIds())
category_ids = coco.getCatIds()
category_names = [category['name'] for category in categories]
print(f'Number of classes: {len(category_ids)}')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Number of classes: 28


In [26]:
num_classes = len(set(category_ids))

In [15]:
image_folder = '/Users/wiesruyters/Documents/WhD/Repositories/NOS_scrape/Datasets/Annotated_img_politicians/images'
image_height, image_width = 256, 256

image_paths = [os.path.join(image_folder, filename) for filename in os.listdir(image_folder)]
resized_images = []

for image_path in image_paths:
    img = cv2.imread(image_path)
    img = cv2.resize(img, (image_width, image_height))
    resized_images.append(img)

resized_images = np.array(resized_images)
print(f'Length of resized_images array: {len(resized_images)}')

Length of resized_images array: 81


In [27]:
annotations = {}
for category_id in category_ids:
    image_ids = coco.getImgIds(catIds=category_id)
    for image_id in image_ids:
        image_info = coco.loadImgs(image_id)[0]
        
        # Retrieve annotations for the selected image
        ann_ids = coco.getAnnIds(imgIds=image_info['id'], catIds=category_id)
        anns = coco.loadAnns(ann_ids)
        
        # Create an entry for the image ID in the dictionary
        if image_info['id'] not in annotations:
            annotations[image_info['id']] = []
        
        # Append the annotations to the image's entry in the dictionary
        annotations[image_info['id']].extend(anns)

# Convert the dictionary of annotations into a list
annotations_list = np.array(list(annotations.values()))

X_train, X_test, Y_train, Y_test = train_test_split(image_paths, annotations_list, test_size=0.2, random_state=42)
Y_train_one_hot = to_categorical(Y_train, num_classes=num_classes)
Y_test_one_hot = to_categorical(Y_test, num_classes=num_classes)

for image_path in image_paths:  # Replace with a list of your image file paths
    img = cv2.imread(image_path)
    img = cv2.resize(img, (image_width, image_height))
    resized_images = np.append(resized_images, [img], axis=0)

resized_images = np.array(resized_images)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (81,) + inhomogeneous part.

In [17]:
model = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 254, 254, 32)      896       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 127, 127, 32)      0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 516128)            0         
                                                                 
 dense_2 (Dense)             (None, 64)                33032256  
                                                                 
 dense_3 (Dense)             (None, 10)                650       
                                                                 
Total params: 33033802 (126.01 MB)
Trainable params: 33033802 (126.01 MB)
Non-trainable params: 0 (0.00 Byte)
__________

In [18]:
model.compile(optimizer=Adam(learning_rate=0.001), loss=categorical_crossentropy, metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=50, validation_data=(X_test, Y_test))

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'str'>"}), (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {\'(<class \\\'dict\\\'> containing {"<class \\\'str\\\'>"} keys and {"<class \\\'int\\\'>", "<class \\\'float\\\'>", \\\'(<class \\\\\\\'list\\\\\\\'> containing values of types {"<class \\\\\\\'float\\\\\\\'>"})\\\', "(<class \\\'list\\\'> containing values of types set())"} values)\'})'})